<a href="https://colab.research.google.com/github/estherdomfeh213/Numpy-Assignment-II/blob/main/Store_Item_Demand_Forecasting_Project_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Store Item Demand Forecasting Challenge
Predict 3 months of item sale at different stores.
The dataset objective is to predict the 3 months of item-level sales data at different store locations
 


In [1]:
! pip install -Uq upgini catboost 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 71.7 MB/s eta 0:00:00


In [3]:
from os.path import exists 
import pandas as pd 

df = pd.read_csv("train.csv")

In [4]:
df.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [5]:
df = df.sample(n=19_000,random_state=0)
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [6]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]


In [7]:
train_features = train.drop(columns=["sales"])
train_target =train["sales"]
test_features = test.drop(columns=["sales"])
test_target = test["sales"]

In [9]:
from upgini import FeaturesEnricher, SearchKey 
from upgini.metadata import CVType 

enricher = FeaturesEnricher(
    search_keys= { 
        "date" : SearchKey.DATE, 
    }, 
    cv = CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features,test_target)])


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Detected task type: ModelTaskType.REGRESSION

Search key country_code `US` was automatically determined by client IP. 
See docs to turn off the automatic detection: https://github.com/upgini/upgini/blob/main/README.md#turn-off-autodetection-for-search-key-columns


Column name,Status,Errors
country_code,All valid,-
target,All valid,-
date,All valid,-



Running search request, search_id=4048c31a-9779-4b37-a7a1-67ab97faef32
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


37 relevant feature(s) found with the search keys: ['date', 'country_code']


,,item,0.609461,100.000000,categorical,
,,store,0.202498,100.000000,categorical,
Upgini,Public data,f_weather_date_weather_umap_48_b39cd0c4,0.031007,100.000000,numerical,Free
Upgini,Public data,f_events_country_date_weekday_9c7b0a2a,0.028711,100.000000,numerical,Free
Upgini,Public data,f_events_date_year_cos1_9014a856,0.025042,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_31_fa6d9a99,0.024426,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_pca_0_d7e0a1fc,0.013810,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_sin1_847b5db1,0.010499,100.000000,numerical,Free
Upgini,Public data,f_events_date_week_cos1_f6a8c1fc,0.008335,100.000000,numerical,Free
Upgini,Public data,f_events_country_date_holiday_state_legal_84d_ahead_f83d666e,0.006467,100.000000,numerical,Free
Upgini,Public data,f_weather_date_weather_umap_34_c3ef5b4f,0.005700,100.000000,numerical,Free


We detected 113 outliers in your sample.
Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Before dropping target outliers size: 19000
After dropping target outliers size: 18887
Calculating accuracy uplift after enrichment...
which makes metrics between the train and eval_set incomparable.

Quality metrics


,Rows,Baseline mean_squared_error,Enriched mean_squared_error,Uplift
,,,,
Train,15148,309.990498,193.497954,116.492544
Eval 1,3739,509.289740,346.434774,162.854966


In [10]:
from catboost import CatBoostRegressor 
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state =0)

enricher.calculate_metrics(
    train_features,train_target,
    eval_set=[(test_features, test_target)],
    estimator= model,
    scoring = "mean_absolute_percentage_error"
)



Calculating accuracy uplift after enrichment...
which makes metrics between the train and eval_set incomparable.


,Rows,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
,,,,
Train,15148,0.255109,0.156666,0.098443
Eval 1,3739,0.270377,0.186325,0.084052


In [11]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)
enriched_train_features.head()
 


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Search key country_code `US` was automatically determined by client IP. 
See docs to turn off the automatic detection: https://github.com/upgini/upgini/blob/main/README.md#turn-off-autodetection-for-search-key-columns


Column name,Status,Errors
country_code,All valid,-
date,All valid,-



Running search request, search_id=7a8dfb04-f6f4-41a5-88e8-1c559f0c4e05
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history
Search key country_code `US` was automatically determined by client IP. 
See docs to turn off the automatic detection: https://github.com/upgini/upgini/blob/main/README.md#turn-off-autodetection-for-search-key-columns


Column name,Status,Errors
country_code,All valid,-
date,All valid,-



Running search request, search_id=ec50d79f-8fee-4435-b24c-79a607672d33
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


,date,store,item,f_weather_date_weather_umap_48_b39cd0c4,f_events_country_date_weekday_9c7b0a2a,f_events_date_year_cos1_9014a856,f_weather_date_weather_umap_31_fa6d9a99,f_weather_date_weather_pca_0_d7e0a1fc,f_events_date_week_sin1_847b5db1,f_events_date_week_cos1_f6a8c1fc,...,f_economic_date_cbpol_umap_4_c5ce4e90,f_economic_date_cpi_umap_9_539b2f30,f_economic_country_date_pcpi_ix_14adc260,f_economic_date_cbpol_pca_9_bde660b4,f_weather_date_weather_umap_13_842ae1fc,f_events_date_year_sin1_3c44bc64,f_weather_date_weather_pca_30_da4d7475,f_economic_date_cbpol_umap_8_36966091,f_weather_date_weather_pca_5_29805999,f_economic_date_cbpol_umap_7_a80d807e
0,2013-01-01,7,5,4.540985,1,0.98522,4.712653,29.676683,0.781831,0.62349,...,4.006054,12.378617,105.579084,-0.332055,3.775319,0.171293,-0.496983,4.843946,-2.88125,3.486583
1,2013-01-01,4,9,4.540985,1,0.98522,4.712653,29.676683,0.781831,0.62349,...,4.006054,12.378617,105.579084,-0.332055,3.775319,0.171293,-0.496983,4.843946,-2.88125,3.486583
2,2013-01-01,1,33,4.540985,1,0.98522,4.712653,29.676683,0.781831,0.62349,...,4.006054,12.378617,105.579084,-0.332055,3.775319,0.171293,-0.496983,4.843946,-2.88125,3.486583
3,2013-01-01,3,41,4.540985,1,0.98522,4.712653,29.676683,0.781831,0.62349,...,4.006054,12.378617,105.579084,-0.332055,3.775319,0.171293,-0.496983,4.843946,-2.88125,3.486583
4,2013-01-01,5,24,4.540985,1,0.98522,4.712653,29.676683,0.781831,0.62349,...,4.006054,12.378617,105.579084,-0.332055,3.775319,0.171293,-0.496983,4.843946,-2.88125,3.486583


In [12]:
model.fit(train_features,  train_target)
preds= model.predict(test_features)
eval_metric(test_target.values, preds,"SMAPE")


[37.65141857448004]

In [13]:
model.fit(enriched_train_features,  train_target)
enriched_preds= model.predict(enriched_test_features)
eval_metric(test_target.values, enriched_preds,"SMAPE")


[14.81461735142678]